In [1]:
# ============================================================
# 02_etl_pyspark.ipynb
# Làm sạch & chuẩn hóa dữ liệu phim trước khi load vào Neo4j
# ============================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

# ------------------------------------------------------------
# 1️⃣ TẠO SPARK SESSION
# ------------------------------------------------------------

spark = SparkSession.builder \
    .appName("MovieLens_ETL_PySpark") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("✅ SparkSession sẵn sàng!")

✅ SparkSession sẵn sàng!


In [2]:
# ------------------------------------------------------------
# 2️⃣ KHAI BÁO ĐƯỜNG DẪN
# ------------------------------------------------------------
DATALAKE_PATH = "../data/datalake/"
WAREHOUSE_PATH = "../data/warehouse/"

os.makedirs(WAREHOUSE_PATH, exist_ok=True)
print(f"📁 Data Warehouse lưu tại: {WAREHOUSE_PATH}")


📁 Data Warehouse lưu tại: ../data/warehouse/


In [3]:
# ------------------------------------------------------------
# 3️⃣ LOAD DỮ LIỆU TỪ DATA LAKE (CSV)
# ------------------------------------------------------------
metadata = spark.read.csv(DATALAKE_PATH + "metadata.csv", header=True)
ratings = spark.read.csv(DATALAKE_PATH + "ratings.csv", header=True)
reviews = spark.read.csv(DATALAKE_PATH + "reviews.csv", header=True)
survey_answers = spark.read.csv(DATALAKE_PATH + "survey_answers.csv", header=True)
tag_count = spark.read.csv(DATALAKE_PATH + "tag_count.csv", header=True)
tags = spark.read.csv(DATALAKE_PATH + "tags.csv", header=True)

print("✅ Đã load dữ liệu từ Data Lake!")

✅ Đã load dữ liệu từ Data Lake!


In [4]:
# ------------------------------------------------------------
# 4️⃣ KIỂM TRA SƠ BỘ SCHEMA & MẪU DỮ LIỆU
# ------------------------------------------------------------

print("\n📌 Schema: metadata")
metadata.printSchema()
metadata.show(5, truncate=False)

print("\n📌 Schema: ratings")
ratings.printSchema()
ratings.show(5, truncate=False)

print("\n📌 Schema: reviews")
reviews.printSchema()
reviews.show(5, truncate=False)

print("\n📌 Schema: survey_answers")
survey_answers.printSchema()
survey_answers.show(5, truncate=False)

print("\n📌 Schema: tag_count")
tag_count.printSchema()
tag_count.show(5, truncate=False)

print("\n📌 Schema: tags")
tags.printSchema()
tags.show(5, truncate=False)

print("\n✅ Hoàn tất kiểm tra schema và mẫu dữ liệu!")


📌 Schema: metadata
root
 |-- avgRating: string (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- directedBy: string (nullable = true)
 |-- imdbId: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- starring: string (nullable = true)
 |-- title: string (nullable = true)

+---------+---------+---------------+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------+----------------------------------+
|avgRating|dateAdded|directedBy     |imdbId |item_id|starring                                                                                                                               |title                             |
+---------+---------+---------------+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------+----------------------------------+
|3.89146  |NULL     |

In [5]:
# ============================================================
# 5️⃣ LÀM SẠCH DỮ LIỆU 
# ============================================================

# Movies
metadata_df = (
    metadata
    .withColumnRenamed("avgRating", "avg_rating")
    .withColumnRenamed("directedBy", "directed_by")
    .withColumnRenamed("starring", "actors")
    .withColumnRenamed("imdbId", "imdb_id")
    .dropna(subset=["item_id", "title"])
    .withColumn("item_id", F.col("item_id").cast("int"))
    .withColumn("avg_rating", F.col("avg_rating").cast("float"))
    .fillna({"directed_by": "Unknown", "actors": "Unknown"})
    .dropDuplicates(["item_id"])
)

# Ratings
ratings_df = (
    ratings
    .dropna(subset=["item_id", "user_id", "rating"])
    .withColumn("item_id", F.col("item_id").cast("int"))
    .withColumn("user_id", F.col("user_id").cast("int"))
    .withColumn("rating", F.col("rating").cast("float"))
    .dropDuplicates(["user_id", "item_id"])
)

# Tags
tags_df = (
    tags
    .dropna(subset=["id", "tag"])
    .withColumn("tag_id", F.col("id").cast("int"))
    .withColumn("tag_name", F.lower(F.col("tag")))
    .withColumn("tag_name", F.regexp_replace("tag_name", r"[^a-z0-9\s]", ""))  # <----
    .withColumn("tag_name", F.trim(F.col("tag_name")))
    .drop("id", "tag")
    .dropDuplicates(["tag_id"])
)

# Reviews
reviews_df = (
    reviews
    .dropna(subset=["item_id", "txt"])
    .withColumn("item_id", F.col("item_id").cast("int"))
    .withColumn("txt", F.trim(F.col("txt")))
    .filter(F.length("txt") > 30)      # <----
    .dropDuplicates(["item_id", "txt"])
)

# Tag Count
tag_count_df = (
    tag_count
    .dropna(subset=["item_id", "tag_id", "num"])
    .withColumn("item_id", F.col("item_id").cast("int"))
    .withColumn("tag_id", F.col("tag_id").cast("int"))
    .withColumn("num", F.col("num").cast("int"))
)

# Survey (loại score = -1)
survey_df = (
    survey_answers
    .dropna(subset=["user_id", "item_id", "tag_id", "score"])
    .withColumn("user_id", F.col("user_id").cast("int"))
    .withColumn("item_id", F.col("item_id").cast("int"))
    .withColumn("tag_id", F.col("tag_id").cast("int"))
    .withColumn("score", F.col("score").cast("int"))
    .filter(F.col("score") > 0)        # <----
)


In [6]:
# ============================================================
# 🔍 KIỂM TRA CHẤT LƯỢNG DỮ LIỆU
# ============================================================

def check_quality(df, df_name, key_cols=None):
    print(f"\n\n===== 🔍 DATA QUALITY CHECK: {df_name} =====")

    # 1) Tổng số dòng + số cột
    print(f"📌 Rows: {df.count()}  |  Columns: {len(df.columns)}")

    # 2) Kiểm tra Null theo từng cột
    print("\n🧩 Missing values per column:")
    df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show(truncate=False)

    # 3) Kiểm tra trùng lặp
    dup_count = df.count() - df.dropDuplicates().count()
    print(f"♻️ Duplicate rows: {dup_count}")

    # 4) Nếu có khoá chính → kiểm tra trùng
    if key_cols:
        print(f"\n🔑 Kiểm tra trùng khóa chính {key_cols}:")
        df.groupBy(key_cols).count().filter("count > 1").show(5, truncate=False)

    print("✅ Done!")

# Gọi hàm cho từng bảng
check_quality(metadata, "metadata", key_cols=["item_id"])
check_quality(ratings, "ratings", key_cols=["user_id", "item_id"])
check_quality(tags, "tags", key_cols=["id"])
check_quality(reviews, "reviews", key_cols=["item_id"])
check_quality(tag_count, "tag_count", key_cols=["item_id", "tag_id"])
check_quality(survey_answers, "survey_answers", key_cols=["user_id", "item_id", "tag_id"])



===== 🔍 DATA QUALITY CHECK: metadata =====
📌 Rows: 84661  |  Columns: 7

🧩 Missing values per column:
+---------+---------+----------+------+-------+--------+-----+
|avgRating|dateAdded|directedBy|imdbId|item_id|starring|title|
+---------+---------+----------+------+-------+--------+-----+
|0        |6518     |3150      |0     |0      |6883    |0    |
+---------+---------+----------+------+-------+--------+-----+

♻️ Duplicate rows: 0

🔑 Kiểm tra trùng khóa chính ['item_id']:
+-------+-----+
|item_id|count|
+-------+-----+
+-------+-----+

✅ Done!


===== 🔍 DATA QUALITY CHECK: ratings =====
📌 Rows: 28490116  |  Columns: 3

🧩 Missing values per column:
+-------+------+-------+
|item_id|rating|user_id|
+-------+------+-------+
|0      |0     |0      |
+-------+------+-------+

♻️ Duplicate rows: 51667

🔑 Kiểm tra trùng khóa chính ['user_id', 'item_id']:
+-------+-------+-----+
|user_id|item_id|count|
+-------+-------+-----+
|606239 |588    |2    |
|953419 |36     |2    |
|251751 |47   

In [7]:
# ------------------------------------------------------------
# 6️⃣ LƯU DỮ LIỆU SẠCH DẠNG PARQUET (Warehouse)
# ------------------------------------------------------------

def save(df, name):
    df.write \
      .mode("overwrite") \
      .option("compression", "snappy") \
      .parquet(f"{WAREHOUSE_PATH}{name}")

save(metadata_df,  "dim_movies")
save(ratings_df,   "fact_ratings")
save(tags_df,      "dim_tags")
save(reviews_df,   "fact_reviews")
save(tag_count_df, "fact_tag_count")
save(survey_df,    "fact_survey")

print("\n🎯 DONE → Data Warehouse đã được tạo thành công!")



🎯 DONE → Data Warehouse đã được tạo thành công!


In [9]:
print("\n📦 Warehouse Storage:")

for folder in os.listdir(WAREHOUSE_PATH):
    full_path = os.path.join(WAREHOUSE_PATH, folder)

    # Bỏ qua file (chỉ xét thư mục)
    if not os.path.isdir(full_path):
        continue

    size = sum(
        os.path.getsize(os.path.join(full_path, f))
        for f in os.listdir(full_path)
    ) / (1024 * 1024)

    print(f" - {folder:20} → {size:8.2f} MB")

print("\n✅ Warehouse kiểm tra hoàn tất!")



📦 Warehouse Storage:
 - dim_movies           →     8.49 MB
 - dim_tags             →     0.02 MB
 - fact_ratings         →   138.11 MB
 - fact_reviews         →  2396.82 MB
 - fact_survey          →     0.17 MB
 - fact_tag_count       →     0.75 MB

✅ Warehouse kiểm tra hoàn tất!


In [11]:
spark.catalog.clearCache()
spark.stop()
print("🛑 Spark Session stopped. RAM has been released.")


🛑 Spark Session stopped. RAM has been released.
